In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkFiles
import os
import pandas as pd

In [2]:
"""----------------------------------------------------------------------------
CREATE SPARK CONTEXT
CREATE SQL CONTEXT
----------------------------------------------------------------------------"""
sc = SparkContext.getOrCreate()
#sc =SparkContext()
sqlContext = SQLContext(sc)

/usr/local/spark/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [37]:
#data_dir="/work/irlin355_1/gratienj/ParallelProgrammingCourse/BigDataHadoopSpark/TPs/data"
#file = os.path.join(data_dir,"iris.csv")
df = pd.read_csv("iris.csv")
panda_df = pd.get_dummies(df)
#panda_df.round()
#print(df_one)
print(panda_df)



     sepal_length  sepal_width  petal_length  petal_width  variety_Setosa  \
0             5.1          3.5           1.4          0.2               1   
1             4.9          3.0           1.4          0.2               1   
2             4.7          3.2           1.3          0.2               1   
3             4.6          3.1           1.5          0.2               1   
4             5.0          3.6           1.4          0.2               1   
..            ...          ...           ...          ...             ...   
145           6.7          3.0           5.2          2.3               0   
146           6.3          2.5           5.0          1.9               0   
147           6.5          3.0           5.2          2.0               0   
148           6.2          3.4           5.4          2.3               0   
149           5.9          3.0           5.1          1.8               0   

     variety_Versicolor  variety_Virginica  
0                     0       

In [38]:
iris_df=sqlContext.createDataFrame(panda_df)
iris_df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety_Setosa: long (nullable = true)
 |-- variety_Versicolor: long (nullable = true)
 |-- variety_Virginica: long (nullable = true)



In [40]:
#Add a numeric indexer for the label/target column
from pyspark.ml.feature import StringIndexer
stringIndexer = StringIndexer(inputCol="variety_Setosa", outputCol="ind_variety")
si_model = stringIndexer.fit(iris_df)

irisNormDf = si_model.transform(iris_df)
irisNormDf.printSchema()
irisNormDf.select("variety_Setosa","ind_variety").distinct().collect()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- variety_Setosa: long (nullable = true)
 |-- variety_Versicolor: long (nullable = true)
 |-- variety_Virginica: long (nullable = true)
 |-- ind_variety: double (nullable = false)



[Row(variety_Setosa=1, ind_variety=1.0),
 Row(variety_Setosa=0, ind_variety=0.0)]

In [41]:
"""--------------------------------------------------------------------------
Perform Data Analytics
-------------------------------------------------------------------------"""

#See standard parameters
irisNormDf.describe().show()


+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|summary|      sepal_length|       sepal_width|      petal_length|       petal_width|     variety_Setosa| variety_Versicolor|  variety_Virginica|        ind_variety|
+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+
|  count|               150|               150|               150|               150|                150|                150|                150|                150|
|   mean| 5.843333333333334|3.0573333333333332|3.7580000000000005|1.1993333333333331| 0.3333333333333333| 0.3333333333333333| 0.3333333333333333| 0.3333333333333333|
| stddev|0.8280661279778632|0.4358662849366984| 1.765298233259466|0.7622376689603464|0.47298376984040214|0.47298376984040214|0.47298376984040214|0.47298376984040214|
|   

In [43]:
"""--------------------------------------------------------------------------
Prepare data for ML
-------------------------------------------------------------------------"""

#Transform to a Data Frame for input to Machine Learing
#Drop columns that are not required (low correlation)

from pyspark.ml.linalg import Vectors
def transformToLabeledPoint(row) :
    lp = ( row["variety_Setosa"], row["ind_variety"], \
                Vectors.dense([row["sepal_length"],\
                        row["sepal_width"], \
                        row["petal_length"], \
                        row["petal_width"]]))
    return lp


irisLp = irisNormDf.rdd.map(transformToLabeledPoint)
irisLpDf = sqlContext.createDataFrame(irisLp,["species","label","features"])

irisLpDf.select("species","label","features").show(50)
irisLpDf.cache()


+-------+-----+-----------------+
|species|label|         features|
+-------+-----+-----------------+
|      1|  1.0|[5.1,3.5,1.4,0.2]|
|      1|  1.0|[4.9,3.0,1.4,0.2]|
|      1|  1.0|[4.7,3.2,1.3,0.2]|
|      1|  1.0|[4.6,3.1,1.5,0.2]|
|      1|  1.0|[5.0,3.6,1.4,0.2]|
|      1|  1.0|[5.4,3.9,1.7,0.4]|
|      1|  1.0|[4.6,3.4,1.4,0.3]|
|      1|  1.0|[5.0,3.4,1.5,0.2]|
|      1|  1.0|[4.4,2.9,1.4,0.2]|
|      1|  1.0|[4.9,3.1,1.5,0.1]|
|      1|  1.0|[5.4,3.7,1.5,0.2]|
|      1|  1.0|[4.8,3.4,1.6,0.2]|
|      1|  1.0|[4.8,3.0,1.4,0.1]|
|      1|  1.0|[4.3,3.0,1.1,0.1]|
|      1|  1.0|[5.8,4.0,1.2,0.2]|
|      1|  1.0|[5.7,4.4,1.5,0.4]|
|      1|  1.0|[5.4,3.9,1.3,0.4]|
|      1|  1.0|[5.1,3.5,1.4,0.3]|
|      1|  1.0|[5.7,3.8,1.7,0.3]|
|      1|  1.0|[5.1,3.8,1.5,0.3]|
|      1|  1.0|[5.4,3.4,1.7,0.2]|
|      1|  1.0|[5.1,3.7,1.5,0.4]|
|      1|  1.0|[4.6,3.6,1.0,0.2]|
|      1|  1.0|[5.1,3.3,1.7,0.5]|
|      1|  1.0|[4.8,3.4,1.9,0.2]|
|      1|  1.0|[5.0,3.0,1.6,0.2]|
|      1|  1.0

DataFrame[species: bigint, label: double, features: vector]

In [44]:
from pyspark.ml.feature import VectorIndexer
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(irisLpDf)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(irisLpDf)

In [45]:
"""--------------------------------------------------------------------------
Perform Machine Learning
-------------------------------------------------------------------------"""

#Split into training and testing data
(trainingData, testData) = irisLpDf.randomSplit([0.9, 0.1])
# trainingData.count()
# testData.count()
# testData.collect()

In [46]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Train a RandomForest model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)

In [15]:
# from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer

# # Convert indexed labels back to original labels.
# labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
#                                labels=labelIndexer.labels)

In [47]:
from pyspark.ml import Pipeline
# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])


In [48]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

In [49]:
# Make predictions.
predictions = model.transform(testData)
predictions.select("prediction","species","label").collect()

[Row(prediction=1.0, species=1, label=1.0),
 Row(prediction=1.0, species=1, label=1.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=1.0, species=1, label=1.0),
 Row(prediction=1.0, species=1, label=1.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0)]

In [51]:
# Select example rows to display.
# predictions.select("predictedLabel", "label", "features").show(5)


In [52]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

Test Error = 0
GBTClassificationModel: uid = GBTClassifier_b7570e8dea68, numTrees=10, numClasses=2, numFeatures=4


In [53]:
#Predict on the test data
predictions.select("prediction","species","label").collect()
#predictions = rfModel.transform(testData)
#predictions.select("prediction","species","label").collect()

[Row(prediction=1.0, species=1, label=1.0),
 Row(prediction=1.0, species=1, label=1.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=1.0, species=1, label=1.0),
 Row(prediction=1.0, species=1, label=1.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0),
 Row(prediction=0.0, species=0, label=0.0)]

In [54]:
#Evaluate accuracy
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", \
                    labelCol="label",metricName="accuracy")
evaluator.evaluate(predictions)  

1.0

In [55]:
#Draw a confusion matrix
predictions.groupBy("label","prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|    4|
|  0.0|       0.0|   13|
+-----+----------+-----+

